In [63]:
import pandas as pd
import numpy as np
import re
final_data = pd.read_csv('final_data.csv')
battle_result = pd.read_csv('battle_result.csv')

In [64]:
final_data

,event_map,battle_type,mode,team 1 brawler 1,team 1 brawler 1 power,team 1 brawler 1 trophies,team 1 brawler 2,team 1 brawler 2 power,team 1 brawler 2 trophies,team 1 brawler 3,...,team 1 brawler 3 trophies,team 2 brawler 1,team 2 brawler 1 power,team 2 brawler 1 trophies,team 2 brawler 2,team 2 brawler 2 power,team 2 brawler 2 trophies,team 2 brawler 3,team 2 brawler 3 power,team 2 brawler 3 trophies
0,Deep Diner,ranked,gemGrab,GUS,11.0,717.0,CROW,11.0,791.0,DARRYL,...,701.0,MORTIS,11.0,791.0,FANG,11.0,740.0,JESSIE,10.0,682.0
1,Sneaky Fields,ranked,brawlBall,RICO,11.0,782.0,NITA,11.0,837.0,EL PRIMO,...,890.0,MORTIS,11.0,799.0,EMZ,10.0,784.0,FRANK,11.0,905.0
2,Sneaky Fields,ranked,brawlBall,POCO,11.0,673.0,JACKY,10.0,681.0,SPIKE,...,704.0,GALE,11.0,651.0,SURGE,11.0,649.0,DARRYL,11.0,752.0
3,Sneaky Fields,ranked,brawlBall,LEON,10.0,641.0,SANDY,10.0,604.0,BIBI,...,663.0,BIBI,11.0,654.0,JACKY,10.0,625.0,CROW,11.0,674.0
4,Sneaky Fields,ranked,brawlBall,COLETTE,7.0,403.0,BUSTER,11.0,419.0,GENE,...,411.0,MAX,8.0,395.0,COLETTE,7.0,390.0,FANG,7.0,446.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256420,Goldarm Gulch,ranked,knockout,JACKY,10.0,829.0,SURGE,10.0,756.0,DARRYL,...,861.0,MORTIS,11.0,791.0,SHELLY,10.0,799.0,RUFFS,10.0,765.0
256421,Goldarm Gulch,ranked,knockout,RICO,11.0,792.0,MORTIS,11.0,800.0,TICK,...,795.0,MAX,11.0,826.0,TICK,10.0,829.0,RICO,11.0,812.0
256422,Goldarm Gulch,ranked,knockout,CROW,11.0,872.0,PIPER,11.0,910.0,STU,...,888.0,BELLE,11.0,899.0,PIPER,10.0,908.0,MAX,11.0,805.0
256423,Goldarm Gulch,ranked,knockout,BEA,11.0,14.0,PENNY,11.0,16.0,FANG,...,16.0,PIPER,11.0,12.0,CARL,11.0,15.0,GRAY,11.0,15.0


In [65]:
# Columns to be linked
linked_columns = [
    'team 1 brawler 1', 'team 1 brawler 2', 'team 1 brawler 3',
    'team 2 brawler 1', 'team 2 brawler 2', 'team 2 brawler 3'
]
# Dictionary to store the mappings
mappings = {}

# Concatenate the linked columns into a single series
concatenated = pd.concat([final_data[col] for col in linked_columns])

# Factorize the concatenated series
factorized_values, unique_values = pd.factorize(concatenated)

# Store the mappings for the linked columns
mappings['Brawlers'] = dict(enumerate(unique_values))

# Apply the factorized values back to the original linked columns
for i, col in enumerate(linked_columns):
    final_data[col] = factorized_values[i * len(final_data):(i + 1) * len(final_data)]

# Enumerate values in each remaining text column and store the mappings
for column in final_data.columns:
    if column not in linked_columns and final_data[column].dtype == 'object':
        final_data[column], unique_values = pd.factorize(final_data[column])
        mappings[column] = dict(enumerate(unique_values))

# Print the first value in the first column
print(final_data.iloc[0, 0])

# Print the mappings
print(mappings)

0
{'Brawlers': {0: 'GUS', 1: 'RICO', 2: 'POCO', 3: 'LEON', 4: 'COLETTE', 5: 'ROSA', 6: 'TICK', 7: 'JANET', 8: 'MR. P', 9: 'NITA', 10: 'EL PRIMO', 11: 'FANG', 12: 'BONNIE', 13: 'MANDY', 14: 'PIPER', 15: 'SPIKE', 16: '8-BIT', 17: 'JESSIE', 18: 'BEA', 19: 'DYNAMIKE', 20: 'BARLEY', 21: 'GRAY', 22: 'ASH', 23: 'STU', 24: 'COLT', 25: 'DARRYL', 26: 'MORTIS', 27: 'FRANK', 28: 'BIBI', 29: 'GRIFF', 30: 'SQUEAK', 31: 'CHESTER', 32: 'GENE', 33: 'MAX', 34: 'JACKY', 35: 'OTIS', 36: 'AMBER', 37: 'EMZ', 38: 'LOU', 39: 'SANDY', 40: 'BUSTER', 41: 'RUFFS', 42: 'PAM', 43: 'GROM', 44: 'BROCK', 45: 'BUZZ', 46: 'SPROUT', 47: 'BO', 48: 'CROW', 49: 'SURGE', 50: 'BULL', 51: 'NANI', 52: 'EVE', 53: 'BYRON', 54: 'SAM', 55: 'SHELLY', 56: 'GALE', 57: 'TARA', 58: 'CARL', 59: 'PENNY', 60: 'BELLE', 61: 'EDGAR', 62: 'LOLA', 63: 'MEG'}, 'event_map': {0: 'Deep Diner', 1: 'Sneaky Fields', 2: 'Flaring Phoenix', 3: 'Pinhole Punt', 4: 'Double Swoosh', 5: 'Open Zone', 6: 'Temple Ruins', 7: 'Super Beach', 8: 'Crystal Arcade', 9:

In [66]:
final_data

,event_map,battle_type,mode,team 1 brawler 1,team 1 brawler 1 power,team 1 brawler 1 trophies,team 1 brawler 2,team 1 brawler 2 power,team 1 brawler 2 trophies,team 1 brawler 3,...,team 1 brawler 3 trophies,team 2 brawler 1,team 2 brawler 1 power,team 2 brawler 1 trophies,team 2 brawler 2,team 2 brawler 2 power,team 2 brawler 2 trophies,team 2 brawler 3,team 2 brawler 3 power,team 2 brawler 3 trophies
0,0,0,0,0,11.0,717.0,48,11.0,791.0,25,...,701.0,26,11.0,791.0,11,11.0,740.0,17,10.0,682.0
1,1,0,1,1,11.0,782.0,9,11.0,837.0,10,...,890.0,26,11.0,799.0,37,10.0,784.0,27,11.0,905.0
2,1,0,1,2,11.0,673.0,34,10.0,681.0,15,...,704.0,56,11.0,651.0,49,11.0,649.0,25,11.0,752.0
3,1,0,1,3,10.0,641.0,39,10.0,604.0,28,...,663.0,28,11.0,654.0,34,10.0,625.0,48,11.0,674.0
4,1,0,1,4,7.0,403.0,40,11.0,419.0,32,...,411.0,33,8.0,395.0,4,7.0,390.0,11,7.0,446.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256420,20,0,2,34,10.0,829.0,49,10.0,756.0,25,...,861.0,26,11.0,791.0,55,10.0,799.0,41,10.0,765.0
256421,20,0,2,1,11.0,792.0,26,11.0,800.0,6,...,795.0,33,11.0,826.0,6,10.0,829.0,1,11.0,812.0
256422,20,0,2,48,11.0,872.0,14,11.0,910.0,23,...,888.0,60,11.0,899.0,14,10.0,908.0,33,11.0,805.0
256423,20,0,2,18,11.0,14.0,59,11.0,16.0,11,...,16.0,14,11.0,12.0,58,11.0,15.0,21,11.0,15.0


In [68]:
# Factorize the values in the column
battle_result['battle_result'], unique_values = pd.factorize(battle_result['battle_result'])

# Print the enumerated battle_result DataFrame
print(battle_result)

# Check how many distinct values there are in the column
distinct_values_count = battle_result['battle_result'].nunique()

# Print the number of distinct values in the column
print(f"\nNumber of distinct values in 'result': {distinct_values_count}")

        battle_result
0                   0
1                   0
2                   0
3                   0
4                   1
...               ...
256420              1
256421              1
256422              1
256423              0
256424              0

[256425 rows x 1 columns]

Number of distinct values in 'result': 2


In [69]:
battle_result

,battle_result
0,0
1,0
2,0
3,0
4,1
...,...
256420,1
256421,1
256422,1
256423,0


In [70]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(final_data, battle_result, test_size=0.2, random_state=42)
X_test, X_eval, y_test, y_eval = train_test_split(X_test, y_test, test_size=0.5, random_state=42)



In [71]:
X_train

,event_map,battle_type,mode,team 1 brawler 1,team 1 brawler 1 power,team 1 brawler 1 trophies,team 1 brawler 2,team 1 brawler 2 power,team 1 brawler 2 trophies,team 1 brawler 3,...,team 1 brawler 3 trophies,team 2 brawler 1,team 2 brawler 1 power,team 2 brawler 1 trophies,team 2 brawler 2,team 2 brawler 2 power,team 2 brawler 2 trophies,team 2 brawler 3,team 2 brawler 3 power,team 2 brawler 3 trophies
207419,5,1,3,26,11.0,856.0,23,11.0,818.0,34,...,725.0,34,9.0,823.0,37,11.0,863.0,57,11.0,795.0
87344,12,0,0,0,7.0,129.0,49,6.0,214.0,21,...,234.0,24,9.0,235.0,40,10.0,114.0,3,6.0,204.0
61108,0,0,0,3,11.0,15.0,45,11.0,17.0,9,...,16.0,10,11.0,17.0,37,11.0,16.0,33,11.0,15.0
66564,5,1,3,9,10.0,617.0,28,9.0,591.0,5,...,652.0,2,10.0,646.0,19,9.0,649.0,25,9.0,640.0
118050,32,0,0,39,9.0,707.0,19,9.0,744.0,28,...,749.0,23,9.0,529.0,48,9.0,641.0,18,9.0,721.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119879,8,2,0,29,10.0,826.0,15,11.0,897.0,48,...,827.0,18,11.0,749.0,37,11.0,914.0,52,10.0,882.0
103694,1,0,1,17,11.0,14.0,59,10.0,13.0,33,...,13.0,49,11.0,14.0,19,11.0,13.0,18,11.0,13.0
131932,1,0,1,26,11.0,778.0,4,10.0,761.0,45,...,724.0,26,9.0,715.0,31,11.0,803.0,28,10.0,739.0
146867,14,2,5,2,9.0,539.0,27,10.0,681.0,25,...,574.0,26,10.0,659.0,24,11.0,660.0,19,9.0,666.0


In [72]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
model = MultinomialNB()
model.fit(X_train, y_train)
y_pred_eval = model.predict(X_test)
accuracy_eval = accuracy_score(y_eval, y_pred_eval)

c:\Users\ellio\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


ValueError: Negative values in data passed to MultinomialNB (input X)

In [16]:
num_unique_values = final_data.iloc[:, 3].nunique()
print(f"Number of different values in the fourth column: {num_unique_values}")

Number of different values in the fourth column: 64


In [3]:
num_nan_event_map = final_data['mode'].isna().sum()
print(f"\nNumber of NaN values in 'event_map': {num_nan_event_map}")


Number of NaN values in 'event_map': 0


In [3]:
import re

# The input string
input_string = "[[{'tag': '#LGC0PQ98', 'name': '🍁PassØut™', 'brawler': {'id': 16000029, 'name': 'BEA', 'power': 11, 'trophies': 14}}, {'tag': '#VULUQUY8', 'name': '金|𝔻ανσχ💔', 'brawler': {'id': 16000019, 'name': 'PENNY', 'power': 11, 'trophies': 16}}, {'tag': '#92CRYR8JR', 'name': 'Adͥℝiͣaͫn🌎⚡', 'brawler': {'id': 16000054, 'name': 'FANG', 'power': 11, 'trophies': 16}}], [{'tag': '#PJJJR9JQ2', 'name': 'schwule gans', 'brawler': {'id': 16000015, 'name': 'PIPER', 'power': 11, 'trophies': 12}}, {'tag': '#YVQQJY0', 'name': 'мєℓιн™', 'brawler': {'id': 16000025, 'name': 'CARL', 'power': 11, 'trophies': 15}}, {'tag': '#9GYCJYGP', 'name': 'Armo', 'brawler': {'id': 16000064, 'name': 'GRAY', 'power': 11, 'trophies': 15}}]]"

# Regular expression to match the desired substrings
pattern = r"'brawler': \{[^}]+\}"

# Find all matches
matches = re.findall(pattern, input_string)

# Store matches in a list
brawler_list = matches

# Print the list
for brawler in brawler_list:
    print(brawler)

'brawler': {'id': 16000029, 'name': 'BEA', 'power': 11, 'trophies': 14}
'brawler': {'id': 16000019, 'name': 'PENNY', 'power': 11, 'trophies': 16}
'brawler': {'id': 16000054, 'name': 'FANG', 'power': 11, 'trophies': 16}
'brawler': {'id': 16000015, 'name': 'PIPER', 'power': 11, 'trophies': 12}
'brawler': {'id': 16000025, 'name': 'CARL', 'power': 11, 'trophies': 15}
'brawler': {'id': 16000064, 'name': 'GRAY', 'power': 11, 'trophies': 15}
